In [ ]:
import os
import numpy as np
from PIL import Image
np.float, np.int = np.float64, np.int_
import subprocess
import numpy as np
from pythreejs import *
from IPython.display import display

DEPTH_WIDTH = 256
DEPTH_HEIGHT = 192
MAX_DEPTH = 20.0
path = 'Data'
confidence = 2
every = 5

def extract_frames(video_path, frame_dir):
    os.makedirs(frame_dir, exist_ok=True)
    
    subprocess.run([
        'ffmpeg', '-i', video_path, '-vsync', 'vfr', '-q:v', '1', '-start_number', '0',
        os.path.join(frame_dir, '%06d.png')
    ])


def quaternion_to_rotation_matrix(quaternion):
    qx, qy, qz, qw = quaternion
    return np.array([
        [1 - 2 * (qy**2 + qz**2), 2 * (qx * qy - qz * qw), 2 * (qx * qz + qy * qw)],
        [2 * (qx * qy + qz * qw), 1 - 2 * (qx**2 + qz**2), 2 * (qy * qz - qx * qw)],
        [2 * (qx * qz - qy * qw), 2 * (qy * qz + qx * qw), 1 - 2 * (qx**2 + qy**2)]
    ])

def get_intrinsics(data):
    fx = data['intrinsics'][0, 0]
    fy = data['intrinsics'][1, 1]
    cx = data['intrinsics'][0, 2]
    cy = data['intrinsics'][1, 2]
    return np.array([[fx * DEPTH_WIDTH / 1920, 0, cx * DEPTH_WIDTH / 1920],
                     [0, fy * DEPTH_HEIGHT / 1440, cy * DEPTH_HEIGHT / 1440],
                     [0, 0, 1]])

def create_point_cloud(rgbd, intrinsics, T_CW):
    rgb, depth = rgbd
    fx, fy = intrinsics[0, 0], intrinsics[1, 1]
    cx, cy = intrinsics[0, 2], intrinsics[1, 2]

    points, colors = [], []
    for v in range(depth.shape[0]):
        for u in range(depth.shape[1]):
            z = depth[v, u]
            if z > 0 and z < MAX_DEPTH:
                x = (u - cx) * z / fx
                y = (v - cy) * z / fy
                point_world = T_CW[:3, :3] @ np.array([x, y, z]) + T_CW[:3, 3]
                points.append(point_world)
                colors.append(rgb[v, u] / 255.0)
    return np.array(points), np.array(colors)

def accumulate_point_cloud(pc, rgbd, intrinsics, T_CW):
    points, colors = create_point_cloud(rgbd, intrinsics, T_CW)
    pc[0].extend(points)
    pc[1].extend(colors)


def visualize_point_cloud(points, colors):
    # Convert colors to the 0-1 range for pythreejs compatibility
    colors = np.clip(colors, 0, 1)
    
    # Create a BufferGeometry with the point cloud data
    geometry = BufferGeometry(
        attributes={
            'position': BufferAttribute(points, normalized=False),
            'color': BufferAttribute(colors, normalized=False)
        }
    )

    # Define the material, use vertex colors for each point
    material = PointsMaterial(vertexColors='VertexColors', size=0.05)
    point_cloud = Points(geometry=geometry, material=material)

    # Set up a scene
    camera = PerspectiveCamera(position=[3, 3, 3], fov=75, children=[DirectionalLight(color='white', position=[0, 0, 1], intensity=0.5)])
    scene = Scene(children=[point_cloud, camera, AmbientLight(color='#777777')])
    
    # Display the scene
    renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=800, height=600)
    display(renderer)

def process_point_clouds(data):
    intrinsics = get_intrinsics(data)
    pc = ([], [])
    video_path = os.path.join(path, 'rgb.mp4')
    frame_dir = os.path.join(path, 'frames')
    
    if not os.path.exists(frame_dir):
        extract_frames(video_path, frame_dir)

    for i, T_WC in enumerate(data['poses']):
        if i % every != 0:
            continue
        print(f"Point cloud {i}", end="\r")
        T_CW = np.linalg.inv(T_WC)
        confidence = np.array(Image.open(os.path.join(path, 'confidence', f'{i:06}.png')))
        depth_path = data['depth_frames'][i]
        depth_m = np.array(Image.open(depth_path)).astype(np.float32) / 1000.0
        depth_m[confidence < confidence] = 0.0
        
        rgb_frame_path = os.path.join(frame_dir, f'{i:06}.png')
        rgb_frame = np.array(Image.open(rgb_frame_path))
        rgbd = (np.array(Image.fromarray(rgb_frame).resize((DEPTH_WIDTH, DEPTH_HEIGHT))), depth_m)
        
        accumulate_point_cloud(pc, rgbd, intrinsics, T_WC)

    points, colors = np.array(pc[0]), np.array(pc[1])
    visualize_point_cloud(points, colors)


intrinsics = np.loadtxt(os.path.join(path, 'camera_matrix.csv'), delimiter=',')
odometry = np.loadtxt(os.path.join(path, 'odometry.csv'), delimiter=',', skiprows=1)
poses = []

for line in odometry:
    position, quaternion = line[2:5], line[5:]
    T_WC = np.eye(4)
    T_WC[:3, :3] = quaternion_to_rotation_matrix(quaternion)
    T_WC[:3, 3] = position
    poses.append(T_WC)

depth_dir = os.path.join(path, 'depth')
depth_frames = [os.path.join(depth_dir, p) for p in sorted(os.listdir(depth_dir))]

process_point_clouds({'poses': poses, 'intrinsics': intrinsics, 'depth_frames': depth_frames})

/opt/anaconda3/envs/Xtractofuse/lib/python3.10/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '
